In [4]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [5]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 95% 81.0M/85.7M [00:02<00:00, 19.9MB/s]
100% 85.7M/85.7M [00:02<00:00, 33.6MB/s]


In [6]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [7]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 4.4 MB/s 
     |████████████████████████████████| 840 kB 78.7 MB/s 
     |████████████████████████████████| 1.2 MB 84.1 MB/s 
     |████████████████████████████████| 87 kB 8.8 MB/s 
     |████████████████████████████████| 596 kB 74.4 MB/s 
     |████████████████████████████████| 1.1 MB 79.6 MB/s 
     |████████████████████████████████| 120 kB 89.0 MB/s 
     |████████████████████████████████| 234 kB 88.6 MB/s 
     |████████████████████████████████| 3.4 MB 77.7 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 6.4 MB 67.1 MB/s 
     |████████████████████████████████| 1.1 MB 68.1 MB/s 
     |████████████████████████████████| 25.3 MB 93.7 MB/s 
     |████████████████████████████████| 99 kB 12.1 MB/s 
     |████████████████████████████████| 352 kB 62.0 MB/s 
     |████████████████████████████████| 47.7 MB 72 kB/s 
     |████████████████████████████████| 462 kB 69.4 MB/s 
     |██████████████

In [8]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [9]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [10]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [11]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [12]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [13]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [14]:
spec = model_spec.get('efficientdet_lite3')

In [16]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=30, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/30
397/397 [==============================] - 242s 502ms/step - det_loss: 0.9765 - cls_loss: 0.7771 - box_loss: 0.0040 - reg_l2_loss: 0.0937 - loss: 1.0702 - learning_rate: 0.0078 - gradient_norm: 3.4216 - val_det_loss: 0.7011 - val_cls_loss: 0.5867 - val_box_loss: 0.0023 - val_reg_l2_loss: 0.0940 - val_loss: 0.7951
Epoch 2/30
397/397 [==============================] - 195s 491ms/step - det_loss: 0.5851 - cls_loss: 0.4858 - box_loss: 0.0020 - reg_l2_loss: 0.0943 - loss: 0.6794 - learning_rate: 0.0074 - gradient_norm: 3.5426 - val_det_loss: 0.4886 - val_cls_loss: 0.4500 - val_box_loss: 7.7264e-04 - val_reg_l2_loss: 0.0945 - val_loss: 0.5831
Epoch 3/30
397/397 [==============================] - 195s 491ms/step - det_loss: 0.5235 - cls_loss: 0.4353 - box_loss: 0.0018 - reg_l2_loss: 0.0946 - loss: 0.6182 - learning_rate: 0.0074 - gradient_norm: 3.3526 - val_det_loss: 0.5626 - val_cls_loss: 0.4762 - val_box_loss: 0.0017 - val_reg_l2_loss: 0.0948 - val_loss: 0.6574
Epoch 4/30
397/397

In [21]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 14.556704759597778


In [22]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 12s 107ms/step

{'AP': 0.8373387, 'AP50': 0.9388482, 'AP75': 0.9021607, 'APs': -1.0, 'APm': 0.5979853, 'APl': 0.8522986, 'ARmax1': 0.7591174, 'ARmax10': 0.8827886, 'ARmax100': 0.88752747, 'ARs': -1.0, 'ARm': 0.6716667, 'ARl': 0.9004663, 'AP_/10': 0.7718218, 'AP_/20': 0.8750044, 'AP_/50': 0.84905607, 'AP_/100': 0.9423837, 'AP_/200': 0.86441165, 'AP_/500': 0.8294986, 'AP_/2000': 0.72919464}


In [23]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 16.869204998016357


In [24]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [25]:
model.export(export_dir='/content/')